In [1]:
from openai import OpenAI

In [2]:
openai_client = OpenAI()

In [3]:
from pathlib import Path

project_path = Path('test-project')
project_path.mkdir(exist_ok=True)

In [6]:
import tools

agent_tools = tools.AgentTools(project_path)

In [7]:
from toyaikit.tools import Tools

tools_obj = Tools()
tools_obj.add_tools(agent_tools)

In [44]:
AGENT_INSTRUCTIONS = """You are a coding agent designed to help with software engineering tasks.

You help users with:
- Writing, editing, and refactoring code
- Debugging and fixing errors
- Explaining code and technical concepts
- Project setup and configuration
- Code reviews and best practices


Your task is to use the available tools to satisfy the requests from the user.

When user asks to implement something, or create something, create and modiry the files
using the provided tools. Use your best judgement when deciding how to name files and folders.

# Core Principles

1. Be concise but thorough - Get straight to the point, but don't skip important details
2. Show your work - Explain your reasoning for complex changes
3. Ask questions - When requirements are unclear, ask rather than assume
4. Use tools effectively - ALWAYS use available tools when relevant
5. Code quality matters - Write clean, maintainable code following best practices

# Working with Code

- Prefer editing over creating new files when possible
- Read before writing - Understand existing patterns before making changes
- Test your changes - Run builds/tests and fix any errors you introduce
- Reference precisely - When pointing to code, use file_path:line_number format

You're here to help users build better software efficiently. Start by understanding what they want to accomplish!
""".strip()

In [12]:
from openai import OpenAI
from toyaikit.llm import OpenAIClient

llm_client = OpenAIClient(client=OpenAI(), model='gpt-4o-mini')

In [13]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner

chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=AGENT_INSTRUCTIONS,
    llm_client=llm_client,
    chat_interface=chat_interface,
)

In [14]:
result = runner.run();

You: create a python script for calculating the sum of all the numbers passed as input


-> Response received


-> Response received


You: run it please 


-> Response received


-> Response received


You: stop


Chat ended.


In [15]:
result.cost

CostInfo(input_cost=Decimal('0.0006153'), output_cost=Decimal('0.000294'), total_cost=Decimal('0.0009093'))

In [16]:
from dataclasses import dataclass

@dataclass
class Skill:
    name: str
    description: str
    content: str

In [21]:
import frontmatter

skills_dir = Path('skills/')

In [22]:
name = 'hello'

In [33]:

class SkillLoader:

    def __init__(self, skills_dir: Path | str = None):
        self.skills_dir = Path(skills_dir)

    def load_skill(self, name: str) -> Skill | None:
        skill_file = self.skills_dir / name / "SKILL.md"
        if not skill_file.exists():
            return None

        parsed = frontmatter.load(skill_file)

        return Skill(
            name=parsed.metadata.get("name", name),
            description=parsed.metadata.get("description", ""),
            content=parsed.content,
        )

    def list_skills(self) -> list[Skill]:
        skills = []

        for skill_dir in sorted(self.skills_dir.iterdir()):
            if not skill_dir.is_dir():
                continue

            skill = self.load_skill(skill_dir.name)
            skills.append(skill)

        return skills

    def get_description(self) -> str:
        skills = self.list_skills()

        skills_listing = "\n".join(
            f"  - {s.name}: {s.description}"
            for s in skills
        )

        return skills_listing

In [34]:
skill_loader = SkillLoader(skills_dir)

In [35]:
print(skill_loader.get_description())

  - coding_standards: Use this when writing or reviewing code - provides project-specific coding standards
  - counter: Count things or list items with numbers
  - deploy_app: Deploy the application using deployment scripts and templates
  - hello: Skill for ALL greeting requests
  - joke: Skill for ALL joke requests


In [36]:
class SkillsTool:
    """Wrapper for the skill loader that exposes skill() as a tool."""

    def __init__(self, skill_loader: SkillLoader):
        self.skill_loader = skill_loader

    def skill(self, name: str) -> dict:
        """Load a skill to get specialized instructions.

        Args:
            name: The exact skill name to load.

        Returns:
            Dictionary with skill information including name, description, and content.
        """
        result = self.skill_loader.load_skill(name)
        return {
            "name": result.name,
            "description": result.description,
            "content": result.content,
        }

In [37]:
skills_tool = SkillsTool(skill_loader)

In [39]:
tools_obj.add_tools(skills_tool)

In [45]:
SKILL_INJECTION_PROMPT = f'''
You have the following skills available which you can load with the skills tool:

{skill_loader.get_description()}
'''.strip()

In [48]:
AGENT_WITH_SKILLS_INSTRUCTIONS = AGENT_INSTRUCTIONS + '\n\n' + SKILL_INJECTION_PROMPT

In [52]:
print(AGENT_WITH_SKILLS_INSTRUCTIONS)

You are a coding agent designed to help with software engineering tasks.

You help users with:
- Writing, editing, and refactoring code
- Debugging and fixing errors
- Explaining code and technical concepts
- Project setup and configuration
- Code reviews and best practices


Your task is to use the available tools to satisfy the requests from the user.

When user asks to implement something, or create something, create and modiry the files
using the provided tools. Use your best judgement when deciding how to name files and folders.

# Core Principles

1. Be concise but thorough - Get straight to the point, but don't skip important details
2. Show your work - Explain your reasoning for complex changes
3. Ask questions - When requirements are unclear, ask rather than assume
4. Use tools effectively - ALWAYS use available tools when relevant
5. Code quality matters - Write clean, maintainable code following best practices

# Working with Code

- Prefer editing over creating new files wh

In [ ]:
runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=AGENT_WITH_SKILLS_INSTRUCTIONS,
    llm_client=llm_client,
    chat_interface=chat_interface,
)

result = runner.run()

In [60]:
@dataclass
class Command:
    name: str
    description: str
    template: str

In [61]:
class CommandLoader:
    def __init__(self, commands_dir: Path | str = None):
        self.commands_dir = Path(commands_dir)

    def load_command(self, name: str) -> Command:
        command_file = self.commands_dir / f"{name}.md"
        if not command_file.exists():
            return None

        parsed = frontmatter.load(command_file, encoding="utf-8")
        metadata = dict(parsed.metadata)

        return Command(
            name=name,
            description=metadata.get("description", ""),
            template=parsed.content,
        )

    def list_commands(self) -> list[Command]:
        """List all available commands."""
        commands = []

        if not self.commands_dir.exists():
            return commands

        for md_file in sorted(self.commands_dir.glob("*.md")):
            name = md_file.stem
            command = self.get(name)
            commands.append(command)

        return commands

In [64]:
command_loader = CommandLoader(Path('commands/'))

In [67]:
def process_template(template: str, arguments: str) -> str:
    # process $1, $2, $3, $ARGUMENTS
    return template


class CommandsTool:
    def __init__(self, command_loader: CommandLoader):
        self.command_loader = command_loader

    def execute_command(self, name: str, arguments: str = "") -> str:
        """Execute a command by name and return the rendered prompt.

        If you see input starting with /command, use this tool to load and execute it.
        If the command doesn't exist, let the user know.

        Examples:
            /deploy -> execute command name="deploy"
            /test -> execute command name="test"

        Args:
            name: The command name (without the / prefix).
            arguments: Optional arguments to substitute into the template.

        Returns:
            The rendered command template as a string.
        """
        if name.startswith('/'):
            name = name.lstrip('/')

        command = self.command_loader.load_command(name)
        if not command:
            return f"Command not found: /{name}"

        return process_template(command.template, arguments)

In [69]:
commands_tool = CommandsTool(command_loader=command_loader)

In [70]:
tools_obj.add_tools(commands_tool)

In [72]:
SKILL_INJECTION_PROMPT = f'''
You have the following skills available which you can load with the skills tool:

{skill_loader.get_description()}

Don't confuse skills and commands:

- Skills are discovered automatically, without user explicitly asking for it
- Instructions to execute commands are given explicitly: "/test" -> "run the 'test' command"

When you see "/command", use the tools to execute the command "command"
'''.strip()

AGENT_WITH_SKILLS_INSTRUCTIONS = AGENT_INSTRUCTIONS + '\n\n' + SKILL_INJECTION_PROMPT

In [74]:
runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=AGENT_WITH_SKILLS_INSTRUCTIONS,
    llm_client=llm_client,
    chat_interface=chat_interface,
)

In [76]:
result = runner.run()

You: /test


-> Response received


-> Response received


You: stop


Chat ended.
